In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import datetime as dt
xrange = range
from pandasql import sqldf
pysqldf = lambda q: sqldf(q, globals())

#pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', 200)

import os
print("The working directory was: {0}".format(os.getcwd()))
os.chdir("C:/hjBoralex/etl/gitcwd")
print("The current working directory is: {0}".format(os.getcwd()))

The working directory was: C:\hjBoralex\etl
The current working directory is: C:\hjBoralex\etl\gitcwd


In [57]:
mypath = "C:/Users/hermann.ngayap/Desktop/data/"
marketpath = "C:/Users/hermann.ngayap/Desktop/data/market/markets/"

## PIPELINE  FICHIER VOLUME MARKET REPOWERING (VMR)

In [67]:
df = pd.read_excel(marketpath + "Volumes marchés_ Repowering.xlsx", sheet_name="BD_temp_1", header=0)

#Out projects
out_projets = ["Bougainville", "Cham Longe 1", "Evits et Josaphats", "Remise Reclainville", 
               "Evits et Josaphats", "Remise Reclainville", "Maurienne / Gourgançon", "La Bouleste", 
               "Cham Longe 1 - off", "Remise Reclainville - off", "Evits et Josaphats - off", "Bougainville - off", 
               "Maurienne / Gourgançon - off", "Saint-André - off"]

df.rename(columns = {"Alias":"projet", "Technologie":"technologie", "COD":"cod", "MW 100%":"mw", "Taux succès":"taux_succès", 
                     "MW pondérés":"puissance_installée", "EOH":"eoh","Mécanisme":"type_hedge", "Début FiT ajusté":"date_debut", 
                     "Date Merchant":"date_merchant"}, inplace = True)

#Drop rows that contain any value in the list and reset index
df = df[df['Parc '].isin(out_projets) == False]
df.reset_index(inplace=True, drop=True)

#To select rows where Nom is NaN
df = df[df['projet'].notna()]
df.reset_index(inplace=True, drop=True)

#To replace correct eolien & solar writing
df["technologie"] = df["technologie"].str.replace("Eolien", "éolien")
df["technologie"] = df["technologie"].str.replace("PV", "solaire")
#
df["puissance_installée"] = df["mw"] * df["taux_succès"]
#
df["date_dementelement"] = df["date_msi"] - pd.DateOffset(months=6)
#
df["en_planif"] = "Non"
#
df = df.assign(asset_id=[1 + i for i in xrange(len(df))])[['asset_id'] + df.columns.tolist()]
df = df.assign(rw_id=[1 + i for i in xrange(len(df))])[['rw_id'] + df.columns.tolist()]
#
df_asset = df[["rw_id","asset_id", "projet_id", "projet", "technologie", "cod", "mw", "taux_succès", "puissance_installée", 
               "type_hedge", "date_debut", "eoh", "date_merchant", "date_dementelement", "repowering", 
               "date_msi", "en_planif"]]

#To select in planif that should be in volume_repowering 
vmr_to_planif = df_asset[df_asset['cod'] > (dt.datetime.today() + pd.offsets.YearEnd()).strftime('%Y-%m-%d')]

vmr_to_planif = vmr_to_planif[["rw_id", "asset_id", "projet_id", "projet", "technologie", "cod", "mw", "taux_succès", 
                               "puissance_installée", "eoh", "date_merchant", "date_dementelement", 
                               "repowering", "date_msi", "en_planif"]]

#To exclude projects in planif from the one in production ones
df_asset = df_asset[df_asset['cod'] <= (dt.datetime.today() + pd.offsets.YearEnd()).strftime('%Y-%m-%d')]

#
hedge_vmr = df_asset[["rw_id", "projet_id", "projet", "technologie", "type_hedge", "cod", 
                      "date_merchant", "date_dementelement", "puissance_installée", "en_planif"]]
#
hedge_vmr = hedge_vmr.assign(hedge_id=[1 + i for i in xrange(len(hedge_vmr))])[['hedge_id'] + hedge_vmr.columns.tolist()]
#
hedge_vmr = hedge_vmr[["rw_id", "hedge_id", "projet_id", "projet", "technologie", "type_hedge", "cod", 
                       "date_merchant", "date_dementelement", "puissance_installée", "en_planif"]]
#
df_asset = df_asset[["rw_id", "asset_id", "projet_id", "projet", "technologie", "cod", "mw", "taux_succès", "puissance_installée", 
                     "eoh", "date_merchant", "date_dementelement", "repowering", "date_msi", "en_planif"]]

#To make export as excel files
vmr_to_planif.to_excel(marketpath + "asset_vmr_to_planif.xlsx", index=False)
df_asset.to_excel(marketpath + "template_asset_vmr.xlsx", index=False)
hedge_vmr.to_excel(marketpath + "hedge_vmr.xlsx", index=False)